<a href="https://colab.research.google.com/github/ommestriker007/Geospatial/blob/main/Forest_fire_Burn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [62]:
# Forest Fire Burn Area Assessment and Regrowth Pattern Analysis
# Study Area: Indonesia Kalimantan Peatland Fires (2019)
# Location: Central Kalimantan, Borneo

import ee
import geemap

In [63]:
ee.Authenticate()
ee.Initialize(project="neon-feat-313910")

In [64]:
AOI = ee.Geometry.Rectangle([112.0, -2.5, 115.0, -0.5])

print("Study Area: Indonesia Kalimantan Peatland Fires (2019)")
print("Location: Central Kalimantan, Borneo")
print("Fire Period: September-October 2019")

Study Area: Indonesia Kalimantan Peatland Fires (2019)
Location: Central Kalimantan, Borneo
Fire Period: September-October 2019


In [65]:
# SENTINEL-2 DATA COLLECTION
# Sentinel-2 Surface Reflectance collection with cloud filtering

S2 = ee.ImageCollection('COPERNICUS/S2_SR') \
        .filterBounds(AOI) \
        .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))

# PRE-FIRE AND POST-FIRE IMAGERY
# Pre-Fire: July-August 2019 (Before fire season)

S2_PreFR = S2.filterDate('2019-06-01', '2019-08-31') \
                .filterBounds(AOI) \
                .sort('CLOUDY_PIXEL_PERCENTAGE') \
                .first()

# Post-Fire: October-November 2019 (After major fires)

S2_PostFR = S2.filterDate('2019-10-15', '2019-11-30') \
                 .filterBounds(AOI) \
                 .sort('CLOUDY_PIXEL_PERCENTAGE') \
                 .first()

print("\nPre-Fire Period: June-August 2019")
print("Post-Fire Period: October-November 2019")


Pre-Fire Period: June-August 2019
Post-Fire Period: October-November 2019


In [66]:
# MULTI-YEAR REGROWTH IMAGERY (2020-2024)
# Using dry season months (July-Oct) for consistency

S2_2020 = S2.filterDate('2020-07-01', '2020-10-30').mean()
S2_2021 = S2.filterDate('2021-07-01', '2021-10-30').mean()
S2_2022 = S2.filterDate('2022-07-01', '2022-10-30').mean()
S2_2023 = S2.filterDate('2023-07-01', '2023-10-30').mean()
S2_2024 = S2.filterDate('2024-07-01', '2024-10-30').mean()

print("\nRegrowth Analysis Years: 2020, 2021, 2022, 2023, 2024")


Regrowth Analysis Years: 2020, 2021, 2022, 2023, 2024


In [67]:
# BURN SEVERITY ANALYSIS (NBR & dNBR)
# Normalized Burn Ratio (NBR)

def getNBR(img):
    """Calculate Normalized Burn Ratio: (NIR - SWIR) / (NIR + SWIR)"""
    return img.normalizedDifference(['B8', 'B12']).rename('NBR')

NBR_Pre = getNBR(S2_PreFR)
NBR_Post = getNBR(S2_PostFR)

# Delta NBR (Burn Severity)
# Higher values = more severe burn

deltaNBR = NBR_Pre.subtract(NBR_Post).rename('deltaNBR')

print("\nBurn Severity Index: delta NBR calculated")
print("Classification: >0.66=High, 0.44-0.66=Moderate-High, 0.27-0.44=Moderate-Low, 0.1-0.27=Low")



Burn Severity Index: delta NBR calculated
Classification: >0.66=High, 0.44-0.66=Moderate-High, 0.27-0.44=Moderate-Low, 0.1-0.27=Low


In [68]:
# VEGETATION RECOVERY ANALYSIS (NDVI)
# Normalized Difference Vegetation Index (NDVI)

def getNDVI(img):
    """Calculate NDVI: (NIR - Red) / (NIR + Red)"""
    return img.normalizedDifference(['B8', 'B4']).rename('NDVI')

NDVI_Pre = getNDVI(S2_PreFR)
NDVI_Post = getNDVI(S2_PostFR)
NDVI_2020 = getNDVI(S2_2020)
NDVI_2021 = getNDVI(S2_2021)
NDVI_2022 = getNDVI(S2_2022)
NDVI_2023 = getNDVI(S2_2023)
NDVI_2024 = getNDVI(S2_2024)

print("\nVegetation Index (NDVI) calculated for all years")


Vegetation Index (NDVI) calculated for all years


In [69]:
# VISUALIZATION SETUP
# Creating interactive map for Kalimantan

Map = geemap.Map(center=[-1.5, 113.5], zoom=8)

# RGB Visualization parameters

visRGB = {
    'bands': ['B4', 'B3', 'B2'],
    'min': 0,
    'max': 3000,
    'gamma': 1.4
}

In [70]:
# False Color (for vegetation and burn scars)
visFalseColor = {
    'bands': ['B8', 'B4', 'B3'],
    'min': 0,
    'max': 3000,
    'gamma': 1.4
}

In [71]:
# NBR Visualization
visNBR = {
    'min': -1,
    'max': 1,
    'palette': ['red', 'yellow', 'white', 'cyan', 'blue']
}

In [72]:
# Delta NBR (Burn Severity) Visualization
visBurnSeverity = {
    'min': -0.1,
    'max': 0.8,
    'palette': ['white', 'yellow', 'orange', 'red', 'darkred']
}

In [73]:
# NDVI Visualization (Vegetation Health)
visNDVI = {
    'min': 0,
    'max': 0.8,
    'palette': ['brown', 'yellow', 'lightgreen', 'green', 'darkgreen']
}

In [74]:
# ADD LAYERS TO MAP
# RGB Imagery - Temporal Comparison
Map.addLayer(S2_PreFR.clip(AOI), visRGB, 'Pre-Fire RGB (Jul-Aug 2019)', False)
Map.addLayer(S2_PostFR.clip(AOI), visRGB, 'Post-Fire RGB (Oct-Nov 2019)', False)
Map.addLayer(S2_2020.clip(AOI), visRGB, 'RGB - 1 Year (2020)', False)
Map.addLayer(S2_2022.clip(AOI), visRGB, 'RGB - 3 Years (2022)', False)
Map.addLayer(S2_2024.clip(AOI), visRGB, 'RGB - 5 Years (2024)', False)

# False Color Composite (Better for vegetation analysis)
Map.addLayer(S2_PreFR.clip(AOI), visFalseColor, 'Pre-Fire False Color', False)
Map.addLayer(S2_PostFR.clip(AOI), visFalseColor, 'Post-Fire False Color', True)

# Burn Severity Analysis
Map.addLayer(NBR_Pre.clip(AOI), visNBR, 'NBR Pre-Fire', False)
Map.addLayer(NBR_Post.clip(AOI), visNBR, 'NBR Post-Fire', False)
Map.addLayer(deltaNBR.clip(AOI), visBurnSeverity, '⚠️ Burn Severity (dNBR)', True)

# NDVI - Vegetation Recovery Timeline
Map.addLayer(NDVI_Pre.clip(AOI), visNDVI, 'NDVI Pre-Fire (2019)', False)
Map.addLayer(NDVI_Post.clip(AOI), visNDVI, 'NDVI Post-Fire (2019)', False)
Map.addLayer(NDVI_2020.clip(AOI), visNDVI, '🌱 NDVI 2020 (1 Year)', False)
Map.addLayer(NDVI_2021.clip(AOI), visNDVI, '🌱 NDVI 2021 (2 Years)', False)
Map.addLayer(NDVI_2022.clip(AOI), visNDVI, '🌱 NDVI 2022 (3 Years)', False)
Map.addLayer(NDVI_2023.clip(AOI), visNDVI, '🌱 NDVI 2023 (4 Years)', False)
Map.addLayer(NDVI_2024.clip(AOI), visNDVI, '🌱 NDVI 2024 (5 Years)', True)

In [75]:
Map.addLayer(AOI, {'color': 'yellow'}, 'Study Area Boundary', True)

print("\n✅ All layers added to map!")
print("\nLayer Guide:")
print("- Toggle layers on/off using the layer control")
print("- Red/Dark colors in dNBR = High burn severity")
print("- Green colors in NDVI = Healthy vegetation")
print("- Compare NDVI across years to see regrowth patterns")

# Display the map
Map


✅ All layers added to map!

Layer Guide:
- Toggle layers on/off using the layer control
- Red/Dark colors in dNBR = High burn severity
- Green colors in NDVI = Healthy vegetation
- Compare NDVI across years to see regrowth patterns


Map(center=[-1.5, 113.5], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI…